<a href="https://colab.research.google.com/github/ayertey01/MachineLearning-Project/blob/main/NLP_Primer_twitter_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *

from tqdm import tqdm
import gc
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample_sub = pd.read_csv('SampleSubmission.csv')
ID_COL, TARGET_COL = sample_sub.columns.tolist()
# display train dataset for possible data cleaning
train.head(10)

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.000000
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000
5,OVNPOAUX,<user> a nearly 67 year old study when mental ...,1.0,0.666667
6,JDA2QDV5,"Study of more than 95,000 kids finds no link b...",1.0,0.666667
7,S6UKR4OJ,psa: VACCINATE YOUR FUCKING KIDS,1.0,1.000000
8,V6IJATBE,Coughing extra on the shuttle and everyone thi...,1.0,0.666667
9,VB25IDQK,AIDS vaccine created at Oregon Health &amp; Sc...,1.0,0.666667


In [ ]:
#find the correlations with agreement in the train data set
train.corr()['agreement']

label        0.138155
agreement    1.000000
Name: agreement, dtype: float64

In [ ]:
train['safe_text'].apply(lambda x: len(x))
train.describe()

,label,agreement
count,10000.000000,9999.000000
mean,0.301567,0.854252
std,0.646718,0.180707
min,-1.000000,0.333333
25%,0.000000,0.666667
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [ ]:
train['label'].value_counts()


 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: label, dtype: int64

In [ ]:
#data cleaning 

In [ ]:
train.loc[~train['label'].isin([0, -1, 1]), 'label'] = -1

In [ ]:
train.isna().sum()

tweet_id     0
safe_text    0
label        0
agreement    2
dtype: int64

In [ ]:
train = train.dropna()
train.isna().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [ ]:
#continue with the test dataset, by displaying and possible cleaning as well
test.head(10)

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."
5,01V1X8XW,I think that active duty soldiers should get v...
6,01VCVYR0,New Choices For Seasonal Flu Vaccines: WebMD H...
7,01VO7KBO,"As if the vaccination scares weren't enough, t..."
8,020SYHIL,Vaccines are Key to Good Health <url> via <url>
9,02CQIJUI,"Mom: ""do you feel sick in any other way? A fev..."


In [ ]:
test.isna().sum()

tweet_id     0
safe_text    1
dtype: int64

In [ ]:
test['safe_text'] = test['safe_text'].fillna('xxxxxx')
test.isna().sum()


tweet_id     0
safe_text    0
dtype: int64

In [ ]:
#now review tweets 
#notice the value of each lable for each instance of tweet

In [ ]:
train[train['label'] == 0]['safe_text'].values[:5]

array(['Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>',
       'Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>',
       "<user> @ this point I have 2 text, butw/Bon Jovi cover playin @ Alibi's hope U can come out 2 MMR BBQ<user> will b there!",
       'My prediction, vaccine exemption in Arizona will end soon. To much money is being lost by big pharma.',
       '1$Mug Noche <user> #mmr #mixmasterrod #dcdj #mmr   @ Mad Hatter <url>'],
      dtype=object)

In [ ]:
train[train['label'] == 1]['safe_text'].values[:5]

array(["I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day",
       '<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?',
       'Study of more than 95,000 kids finds no link between MMR vaccine and autism <url>',
       'psa: VACCINATE YOUR FUCKING KIDS',
       'Coughing extra on the shuttle and everyone thinks I have the measles. 😂 #VaccinateYourKids'],
      dtype=object)

In [ ]:
train[train['label'] == -1]['safe_text'].values[:5]

array(['#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD',
       "I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",
       '<user> #CDC lied and hid data that black boys have an 340% uncreased risk of developing autism after MMR #CDCwhistleblower help!?!?',
       '<user> vaccines causing autism',
       '<user> <user> Other than that, his defense is not against vaccines being harmful - but the American life being unhealthy.'],
      dtype=object)

**Notice the lable values for each of the three instance, depending on the value of label the tweet is classified as being Neutral,Positive and Negative, that is 0,1,-1 respectively.**

In [ ]:
#data modeling
tmp = pd.DataFrame()
tmp['text'] = train['safe_text']
tmp['labels'] = train['label']
tmp_test = test[['safe_text']].rename({'safe_text': 'text'}, axis=1)
tmp_test['labels'] = 0
tmp_trn, tmp_val = train_test_split(tmp, test_size=0.15, random_state=2)

In [ ]:
def create_model(model_type, model_name, n_epochs = 2, train_batch_size = 112, eval_batch_size = 144, seq_len = 134, lr = 2e-5):
  transform = ClassificationModel(model_type, model_name,num_labels=1, args={'train_batch_size':train_batch_size,
                                                                         "eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': n_epochs,
                                                                         'max_seq_length': seq_len,
                                                                         'regression': True,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":lr,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,})
  return transform

In [ ]:
transform =create_model('roberta', 'roberta-base', n_epochs=3)
transform.train_model(tmp_trn)
preds_val = transform.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = transform.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_1 = preds_val
pt_1 = test_preds

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/8499 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/76 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/11 [00:00<?, ?it/s]

RMSE: 0.527519154954871


  0%|          | 0/5177 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
transform = create_model('roberta', 'roberta-large', n_epochs=1, train_batch_size=16, eval_batch_size=16)
transform.train_model(tmp_trn)
preds_val = transform.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = transform.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_2 = preds_val
pt_2 = test_preds

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

  0%|          | 0/8499 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/532 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/94 [00:00<?, ?it/s]

RMSE: 0.4916767996260645


  0%|          | 0/5177 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/324 [00:00<?, ?it/s]

In [ ]:
transform = create_model('roberta', 'roberta-large', n_epochs=2, train_batch_size=16, eval_batch_size=16, lr = 2e-5)
transform.train_model(tmp_trn)
preds_val = transform.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = transform.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_3 = preds_val
pt_3 = test_preds

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

  0%|          | 0/8499 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/532 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/532 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/94 [00:00<?, ?it/s]

In [ ]:
transform = create_model('roberta', 'roberta-large', n_epochs=3, train_batch_size=16, eval_batch_size=16, lr = 1e-5)
transform.train_model(tmp_trn)
preds_val = transform.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, -1, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = transform.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, -1, 1)
pv_4 = preds_val
pt_4 = test_preds




In [ ]:
pv = ((pv_1 * 0.3 + pv_2 * 0.7) * 0.3 + pv_3 * 0.7) * 0.65 + pv_4 * 0.35
print(f"RMSE: {mse(tmp_val['labels'], pv)**0.5}")

In [ ]:
#pt_1_reshaped = pt_1[:1500]
#pt_2_reshaped = pt_2[:1500]
#pt_4_reshaped = pt_4[:1500]
tp = (((pt_1 * 0.3 + pt_2 * 0.7) * 0.3 + pt_3 * 0.7) * 0.65) + (pt_4 * 0.35)

In [ ]:
pd.Series(tp).describe()

In [ ]:
# Assign model predictions to a new variable
final_test_preds = tp

# Create a new dataframe with ID_COL and TARGET_COL columns
preds_df_final = pd.DataFrame({
    ID_COL: test[ID_COL][:1500],
    TARGET_COL: final_test_preds
})

# Export dataframe to a CSV file
SUBMISSION_FILE = 'predictions_sample.csv'
preds_df_final.to_csv(SUBMISSION_FILE, index=False)

In [ ]:

preds_df_final.head()

In [ ]:

preds_df_final.tail()

In [ ]:
preds_df_final['label'].describe()

In [ ]:
preds_df_final.isna().sum()

In [ ]:
from google.colab import files
files.download(SUBMISSION_FILE)